In [64]:
import pandas as pd

In [65]:
scn1_event_data= pd.read_csv('Scenario1/Actual User Event_mod.csv')
scn1_event_data.head()

Activity                         Time  Application/Device
0         ReTweet  20-Mar-2020 09:40:42 +05:30  Twitter for iPhone
1  Reply To Tweet  20-Mar-2020 09:40:36 +05:30  Twitter for iPhone
2         ReTweet  20-Mar-2020 09:35:50 +05:30  Twitter for iPhone
3           Tweet  20-Mar-2020 09:35:41 +05:30  Twitter for iPhone
4         ReTweet  19-Mar-2020 06:40:22 +05:30  Twitter for iPhone

In [66]:
print(scn1_event_data.Activity.value_counts())

scn1_event_data.Activity.replace('ReTweet', "Tweet With Text", inplace = True)
scn1_event_data.Activity.replace('Reply To Tweet', "Tweet With Text", inplace = True)
scn1_event_data.Activity.replace('Tweet', "Tweet With Text", inplace = True)

print(scn1_event_data.Activity.value_counts())

ReTweet             568
Tweet With Image     95
Reply To Tweet       76
Tweet With Video     66
Tweet                55
Name: Activity, dtype: int64
Tweet With Text     699
Tweet With Image     95
Tweet With Video     66
Name: Activity, dtype: int64


In [67]:
from datetime import datetime

def get_minute(lstr_datetime):
    lstr_datetime = lstr_datetime.rstrip('+05:30').strip()
    date_time_obj = datetime.strptime(lstr_datetime, '%d-%b-%Y %H:%M:%S')
    return date_time_obj.strftime("%d-%b-%Y %H:%M")

In [68]:
scn1_event_data['Minute'] = pd.to_datetime(scn1_event_data['Time'].apply(lambda x: get_minute(x)))

In [69]:
start_date = scn1_event_data.Minute.min()
end_date = scn1_event_data.Minute.max()

In [70]:
scn1_event_data.head()

Activity                         Time  Application/Device  \
0  Tweet With Text  20-Mar-2020 09:40:42 +05:30  Twitter for iPhone   
1  Tweet With Text  20-Mar-2020 09:40:36 +05:30  Twitter for iPhone   
2  Tweet With Text  20-Mar-2020 09:35:50 +05:30  Twitter for iPhone   
3  Tweet With Text  20-Mar-2020 09:35:41 +05:30  Twitter for iPhone   
4  Tweet With Text  19-Mar-2020 06:40:22 +05:30  Twitter for iPhone   

               Minute  
0 2020-03-20 09:40:00  
1 2020-03-20 09:40:00  
2 2020-03-20 09:35:00  
3 2020-03-20 09:35:00  
4 2020-03-19 06:40:00

<hr>

In [71]:
scn1_sensor_data = pd.read_csv('Scenario1/Sensor Data.csv')
scn1_sensor_data.head()

Activity Type                         Time  User Identity
0   Tweet With Text  24-Jan-2020 21:22:00 +05:30      812218221
1   Tweet With Text  24-Jan-2020 21:24:00 +05:30      812218221
2   Tweet With Text  24-Jan-2020 21:26:00 +05:30      812218221
3  Tweet With Image  24-Jan-2020 21:42:00 +05:30      812218221
4   Tweet With Text  24-Jan-2020 21:43:00 +05:30      812218221

In [72]:
scn1_sensor_data['Minute'] = scn1_sensor_data['Time'].apply(lambda x: get_minute(x))

In [73]:
scn1_sensor_data['Minute'] = pd.to_datetime(scn1_sensor_data['Minute'])

In [74]:
scn1_sensor_data.head()

Activity Type                         Time  User Identity  \
0   Tweet With Text  24-Jan-2020 21:22:00 +05:30      812218221   
1   Tweet With Text  24-Jan-2020 21:24:00 +05:30      812218221   
2   Tweet With Text  24-Jan-2020 21:26:00 +05:30      812218221   
3  Tweet With Image  24-Jan-2020 21:42:00 +05:30      812218221   
4   Tweet With Text  24-Jan-2020 21:43:00 +05:30      812218221   

               Minute  
0 2020-01-24 21:22:00  
1 2020-01-24 21:24:00  
2 2020-01-24 21:26:00  
3 2020-01-24 21:42:00  
4 2020-01-24 21:43:00

In [75]:
scn1_sensor_data['Activity Type'].value_counts()

Tweet With Text     741368
Tweet With Image    106982
Tweet With Video     11364
Tweet With GIF        8773
Name: Activity Type, dtype: int64

In [76]:
llst_df = []
for event_Minute in scn1_event_data['Minute'].unique():
    ldf_row = scn1_event_data[scn1_event_data['Minute'] == event_Minute]
    ldct_row = {}
    ldct_row["Event_Activity"] = ldf_row["Activity"]
    ldf_sensor_data_minute_wise = scn1_sensor_data[scn1_sensor_data['Minute'] == list(ldf_row['Minute'])[0]]
    ldf_sensor_data_minute_wise = ldf_sensor_data_minute_wise[ldf_sensor_data_minute_wise['Activity Type'] == list(ldf_row['Activity'])[0]]
    ldf_sensor_data_minute_wise['Event_Time'] = list(ldf_row['Time'])[0]
    llst_df.append(ldf_sensor_data_minute_wise)

In [77]:
from collections import Counter 

llst_global = []
for uniq_id in scn1_sensor_data['User Identity'].unique():
    llst_id_wise_entry = []
    llst_id_wise_entry.append(uniq_id)
    
    no_of_sensor_events = 0
    no_of_occurences = 0
    
    for sensor_event in llst_df:
        
        llst_ids = sensor_event['User Identity'].values.tolist()
        
        if llst_ids:
            if uniq_id in llst_ids:
                
                no_of_sensor_events = no_of_sensor_events + 1
                
                d = Counter(llst_ids)
                no_of_occurences = no_of_occurences + d[uniq_id]
    
    llst_id_wise_entry.append(no_of_sensor_events)
    llst_id_wise_entry.append(no_of_occurences)
    llst_id_wise_entry.append(len(scn1_sensor_data[scn1_sensor_data['User Identity'] == uniq_id]))
    llst_global.append(llst_id_wise_entry)

In [78]:

ldf_final_outcome = pd.DataFrame(llst_global, columns = 
                                 ['UId','No_of_Dfs','Overlapped_Occurences',
                                  'Total_No_of_Occurences']).sort_values(by='Overlapped_Occurences', ascending = False)

ldf_final_outcome['Normalized'] = [(float(i))/max(ldf_final_outcome['Overlapped_Occurences']) 
                                    for i in ldf_final_outcome['Overlapped_Occurences']]

ldf_final_outcome['Probability'] = ldf_final_outcome.apply(lambda x: 
                                                (x['Overlapped_Occurences']/x['Total_No_of_Occurences'])*100, axis = 1)


ldf_final_outcome = ldf_final_outcome.sort_values(by='Probability', ascending = False)
ldf_final_outcome.head(n=100)

UId  No_of_Dfs  Overlapped_Occurences  Total_No_of_Occurences  \
101     555002222         42                     42                     349   
119     560588532         20                     20                     993   
111     541187967         45                     45                    2446   
140     546871343         32                     32                    1836   
145  831023626854          8                      8                     468   
4       545472421         40                     40                    2423   
53   831042684836         29                     29                    1798   
114  831044709394         35                     35                    2237   
59      543444435         33                     33                    2289   
88      547816061         29                     29                    2020   
138  831021519489         25                     25                    1873   
105     508546785         26                     26                    2067   
100     531999047         25                     25                    2229   
32      564888050         33                     33                    2987   
136  831040166588         28                     28                    2541   
26   831022012657         25                     25                    2271   
89      554771243         24                     24                    2301   
139     560792439         33                     33                    3198   
99      505634599         18                     18                    1746   
68      555941463         23                     23                    2253   
19      500900590         27                     27                    2714   
109     552672788         22                     22                    2276   
115     503544109         35                     35                    3751   
61      502973809         32                     32                    3445   
70      555294441         24                     24                    2588   
77      504100776         28                     28                    3124   
65      562111339         18                     18                    2032   
15      550183065         38                     38                    4329   
123     536777227          9                      9                    1041   
5       546783733         34                     34                    3996   
..            ...        ...                    ...                     ...   
80   831020189705         28                     28                    4605   
69      555129453         19                     19                    3185   
50      553535379         46                     46                    7733   
51      548816078         26                     26                    4411   
82      558129192         23                     23                    3904   
63      554298118         29                     29                    4926   
0       812218221         36                     36                    6121   
112     505559666         19                     19                    3252   
93      550143131         30                     30                    5150   
113     535517744         20                     20                    3438   
31      564258554         30                     30                    5211   
129     552217999         28                     28                    4893   
104  831023953634         37                     37                    6509   
120     555579565         17                     17                    3003   
132     562555252         22                     22                    3985   
60      563867139         23                     23                    4252   
106     557322299         18                     18                    3394   
133     554159503         45                     45                    8506   
3       541968427         17                     17             